In [40]:
import re
import pandas as pd
import numpy as np

In [41]:
temp_one_part_l = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/1754.csv")

In [3]:
# Sanity check: We make sure the diferrence between timestamps is not more or less than expected
temp_one_part_l['time_indicator'] = temp_one_part_l['index'].diff()
temp_one_part_l[(temp_one_part_l['time_indicator'] > 1) | (temp_one_part_l['time_indicator'] < -1)]

,Unnamed: 0,index,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,...,bodyTrackerRotation.y,bodyTrackerRotation.z,hitColliderType,hitObjectColliderBoundsCenter,hitObjectColliderName,ordinalOfHit,hitPointOnObject_x,hitPointOnObject_y,hitPointOnObject_z,time_indicator
547,553,133844,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -99.08572387695312...,...,344.072144,359.920959,UnityEngine.SphereCollider,"{'x': -98.7122573852539, 'y': 5.82877254486084...",CollisionObject1,1.0,-99.085724,4.944987,212.253250,7.0
1391,1431,134722,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0,[{'hitPointOnObject': {'x': -59.36338424682617...,...,91.363693,5.718540,UnityEngine.MeshCollider,"{'x': 39.25787353515625, 'y': -4.4083013534545...",road_base_network.004,1.0,-59.363384,0.466697,33.402351,35.0
1642,1686,134977,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -90.65501403808594...,...,114.432625,6.266405,UnityEngine.SphereCollider,"{'x': -88.05426788330078, 'y': 12.470046997070...",CollisionObject2,1.0,-90.655014,11.352463,-51.465061,5.0
1650,1696,134987,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -89.92011260986328...,...,118.613625,6.692514,UnityEngine.SphereCollider,"{'x': -88.01226043701172, 'y': 6.0187730789184...",CollisionObject1,1.0,-89.920113,8.475922,-49.828369,3.0
1675,1723,135014,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -85.83135986328125...,...,123.968521,6.519924,UnityEngine.BoxCollider,"{'x': -86.88899993896484, 'y': 19.279701232910...",crane_1,1.0,-85.831360,18.660173,-27.485453,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962056,1980777,131582,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,[{'hitPointOnObject': {'x': 226.59303283691406...,...,11.873497,4.811445,UnityEngine.MeshCollider,"{'x': 226.93492126464844, 'y': 5.1365947723388...",TaskBuilding_14,1.0,226.593033,9.736407,123.059998,11.0
1962062,1980790,131595,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 224.813720703125, ...",...,11.535590,4.782461,UnityEngine.MeshCollider,"{'x': 226.93492126464844, 'y': 5.1365947723388...",TaskBuilding_14,1.0,224.813721,8.808421,117.307022,8.0
1962244,1980978,131783,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 295.0853576660156,...",...,11.548946,4.774898,UnityEngine.MeshCollider,"{'x': 299.42376708984375, 'y': 12.845506668090...",Windmill-TaskBuilding_10_1,1.0,295.085358,18.384865,186.220657,7.0
1963222,1981988,132793,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 83.76455688476562,...",...,306.395264,4.704519,UnityEngine.BoxCollider,"{'x': 83.68951416015625, 'y': 4.23225212097168...",Lamppost_v1 (26),1.0,83.764557,6.882790,28.000019,33.0


In [5]:
temp_one_part_l.columns

Index(['Unnamed: 0', 'index', 'SubjectID', 'Session', 'SessionSubsection',
       'timeStampDataPointStart', 'timeStampDataPointEnd',
       'timeStampGetVerboseData', 'combinedGazeValidityBitmask',
       'rayCastHitsCombinedEyes', 'eyePositionCombinedWorld.x',
       'eyePositionCombinedWorld.y', 'eyePositionCombinedWorld.z',
       'eyeDirectionCombinedWorld.y', 'eyeDirectionCombinedWorld.z',
       'eyeDirectionCombinedLocal.x', 'eyeDirectionCombinedLocal.y',
       'eyeDirectionCombinedLocal.z', 'hmdPosition.x', 'hmdPosition.y',
       'hmdPosition.z', 'hmdDirectionForward.x', 'hmdDirectionForward.y',
       'hmdDirectionForward.z', 'hmdRotation.x', 'hmdRotation.y',
       'hmdRotation.z', 'hmdDirectionUp.x', 'hmdDirectionUp.y',
       'hmdDirectionUp.z', 'playerBodyPosition.x', 'playerBodyPosition.y',
       'playerBodyPosition.z', 'bodyTrackerPosition.x',
       'bodyTrackerPosition.y', 'bodyTrackerPosition.z',
       'bodyTrackerRotation.x', 'bodyTrackerRotation.y',
       'bod

In [42]:
#Since we have two colliders hits per frame, we calculate the distance between each hit and the participant, this tells us which object was closest to the participant
temp_one_part_l['Eucledian_distance'] = np.linalg.norm(temp_one_part_l.loc[:, ["hitPointOnObject_x","hitPointOnObject_y","hitPointOnObject_z"]].values - temp_one_part_l.loc[:, ["playerBodyPosition.x","playerBodyPosition.y","playerBodyPosition.z"]], axis=1)

In [31]:
# Since the collider names are too detailed, here we create a dictionary with patterns to classify them into our categories of interest
# INCLUDE LANDMARKS
patterns = {'\d{2}_Sa':'Passive_Agent', '\d{2}_Cma':'Active_Agent', 'Building_\d+': 'Building', 'Graffity_\d+': 'Graffity'}
patterns.update(dict.fromkeys(['TaskBuilding_2','TaskBuilding_3', 'TaskBuilding_5', 'TaskBuilding_8', 'TaskBuilding_9', 'TaskBuilding_11', 'TaskBuilding_13', 'TaskBuilding_14', 'TaskBuilding_20', 'TaskBuilding_21', 'TaskBuilding_23','TaskBuilding_27', 'TaskBuilding_29', 'TaskBuilding_32', 'TaskBuilding_34',  'TaskBuilding_38', 'TaskBuilding_41', 'TaskBuilding_42', 'TaskBuilding_44', 'TaskBuilding_45', 'TaskBuilding_47', 'TaskBuilding_50', 'TaskBuilding_51', 'TaskBuilding_52'], 'TaskBuilding_Public'))
patterns.update(dict.fromkeys(['TaskBuilding_1','TaskBuilding_4', 'TaskBuilding_6', 'TaskBuilding_7', 'TaskBuilding_12', 'TaskBuilding_15', 'TaskBuilding_17', 'TaskBuilding_18', 'TaskBuilding_19', 'TaskBuilding_22', 'TaskBuilding_24','TaskBuilding_25', 'TaskBuilding_26', 'TaskBuilding_28', 'TaskBuilding_30',  'TaskBuilding_31', 'TaskBuilding_33', 'TaskBuilding_35', 'TaskBuilding_36', 'TaskBuilding_37', 'TaskBuilding_39', 'TaskBuilding_40', 'TaskBuilding_43', 'TaskBuilding_48', 'TaskBuilding_54' ], 'TaskBuilding_Residential'))
default_val = 'Background'

In [35]:
# Here we look for the patterns contained in the dictionary and create the more general/informative variable Collider_Categorical
temp_one_part_l['Collider_Categorical'] = temp_one_part_l['hitObjectColliderName'].apply(lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))

In [ ]:
# Since the double hits where unfolded in a long format in case of more than one hit per frame the second hit would be the row directly underneath.
# Therefore, if we want to know which hit was closest to the participant per frame we can compare each row with the previous one and see which one has the smallest Euclidean distance.
temp_one_part_l['Previous_Euclidean_value'] = temp_one_part_l['Euclidean_distance'].shift(1)
#Here we declare the conditions to choose between collider hits:
temp_one_part_l['Collider_stays'] =  (temp_one_part_l['Euclidean_distance'] <  temp_one_part_l['Previous_Euclidean_value']) & (temp_one_part_l["ordinalOfHit"] == 2)

In [21]:
sample = temp_one_part_l.sample(n=5000)

In [24]:
sample.head()

,Unnamed: 0,index,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,...,hitObjectColliderBoundsCenter,hitObjectColliderName,ordinalOfHit,hitPointOnObject_x,hitPointOnObject_y,hitPointOnObject_z,time_indicator,Eucledian_distance,Previous_value,Collider_stays
22448,22646,155937,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': 7.7885918617248535...,...,"{'x': 9.010000228881836, 'y': -0.1894607543945...",Building_124,1.0,7.788592,-2.037132,-154.861877,1.0,10.511569,55.783667,False
1464916,1480847,1614138,1754,4,3,1.647258e+09,1.647258e+09,1.647258e+09,3,"[{'hitPointOnObject': {'x': -294.946044921875,...",...,"{'x': -304.0008239746094, 'y': 11.011278152465...",Building_210,1.0,-294.946045,4.959328,-75.312897,1.0,111.907342,239.221413,False
1175050,1189225,1322516,1754,4,1,1.647256e+09,1.647256e+09,1.647256e+09,3,[{'hitPointOnObject': {'x': -24.51759338378906...,...,"{'x': -21.745500564575195, 'y': 3.280989170074...",CollisionObject1,1.0,-24.517593,2.048740,-29.883991,1.0,71.465483,102.448532,False
1157389,1171346,1304637,1754,3,3,1.647012e+09,1.647012e+09,1.647012e+09,3,[{'hitPointOnObject': {'x': -114.0694503784179...,...,"{'x': -107.4800033569336, 'y': 5.9338560104370...",Building_163,1.0,-114.069450,3.098175,83.584488,1.0,9.410727,20.316714,False
1132612,1146301,1279592,1754,3,3,1.647012e+09,1.647012e+09,1.647012e+09,3,"[{'hitPointOnObject': {'x': 73.96194458007812,...",...,"{'x': 58.28839111328125, 'y': 1.17256546020507...",Building_133,2.0,66.549194,-3.163616,-164.990067,1.0,36.166028,25.986278,False


In [32]:
# create the new column using a dictionary comprehension
sample['Collider_Categorical'] = sample['hitObjectColliderName'].apply(lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))

In [36]:
temp_one_part_l.groupby("Collider_Categorical")["ordinalOfHit"].count().sort_values()

Collider_Categorical
Passive_Agent                  6786
Active_Agent                   9415
TaskBuilding_Residential      51066
Graffity                      74584
TaskBuilding_Public          180464
Building                     497496
Background                  1143907
Name: ordinalOfHit, dtype: int64

In [39]:
temp_one_part_l.tail(100)

,Unnamed: 0,index,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,...,hitObjectColliderName,ordinalOfHit,hitPointOnObject_x,hitPointOnObject_y,hitPointOnObject_z,time_indicator,Eucledian_distance,Previous_value,Collider_stays,Collider_Categorical
1963618,1982386,133191,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 99.7020492553711, ...",...,Building_108,1.0,99.702049,0.114462,11.029458,1.0,5.114291,16.796816,False,Building
1963619,1982387,133192,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 99.7020492553711, ...",...,terrain_I.001,2.0,89.438225,-1.233989,5.832491,1.0,16.521456,5.114291,False,Background
1963620,1982388,133193,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 99.7020492553711, ...",...,Building_108,1.0,99.702049,0.114462,11.029458,1.0,5.114291,16.521456,False,Building
1963621,1982389,133194,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 99.7020492553711, ...",...,terrain_I.001,2.0,89.438225,-1.233989,5.832491,1.0,16.521456,5.114291,False,Background
1963622,1982390,133195,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 99.6587905883789, ...",...,Building_108,1.0,99.658791,0.111696,11.082476,1.0,5.112534,16.521456,False,Building
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963713,1982481,133286,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 54.16472244262695,...",...,TaskBuilding_30,2.0,40.996479,4.011209,59.922562,1.0,76.989031,60.656410,False,TaskBuilding_Public
1963714,1982482,133287,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",...,TaskBuilding_30,1.0,39.853935,3.679493,60.072559,1.0,77.917021,76.989031,False,TaskBuilding_Public
1963715,1982483,133288,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",...,Building_99,2.0,36.397789,3.831103,62.565891,1.0,82.180532,77.917021,False,Building
1963716,1982484,133289,1754,5,3,1.647345e+09,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",...,TaskBuilding_30,1.0,39.853935,3.679493,60.072559,1.0,77.917021,82.180532,False,TaskBuilding_Public


In [33]:
sample.head(300)

,Unnamed: 0,index,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,...,hitObjectColliderName,ordinalOfHit,hitPointOnObject_x,hitPointOnObject_y,hitPointOnObject_z,time_indicator,Eucledian_distance,Previous_value,Collider_stays,Collider_Categorical
22448,22646,155937,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': 7.7885918617248535...,...,Building_124,1.0,7.788592,-2.037132,-154.861877,1.0,10.511569,55.783667,False,Building
1464916,1480847,1614138,1754,4,3,1.647258e+09,1.647258e+09,1.647258e+09,3,"[{'hitPointOnObject': {'x': -294.946044921875,...",...,Building_210,1.0,-294.946045,4.959328,-75.312897,1.0,111.907342,239.221413,False,Building
1175050,1189225,1322516,1754,4,1,1.647256e+09,1.647256e+09,1.647256e+09,3,[{'hitPointOnObject': {'x': -24.51759338378906...,...,CollisionObject1,1.0,-24.517593,2.048740,-29.883991,1.0,71.465483,102.448532,False,Background
1157389,1171346,1304637,1754,3,3,1.647012e+09,1.647012e+09,1.647012e+09,3,[{'hitPointOnObject': {'x': -114.0694503784179...,...,Building_163,1.0,-114.069450,3.098175,83.584488,1.0,9.410727,20.316714,False,Building
1132612,1146301,1279592,1754,3,3,1.647012e+09,1.647012e+09,1.647012e+09,3,"[{'hitPointOnObject': {'x': 73.96194458007812,...",...,Building_133,2.0,66.549194,-3.163616,-164.990067,1.0,36.166028,25.986278,False,Building
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86710,87957,221248,1754,1,1,1.646741e+09,1.646741e+09,1.646741e+09,3,[{'hitPointOnObject': {'x': 203.59976196289062...,...,terrain_U_LP.001,2.0,184.968185,-1.665959,154.784393,1.0,23.065174,4.177603,False,Background
639537,646928,780219,1754,2,2,1.646920e+09,1.646920e+09,1.646920e+09,3,[{'hitPointOnObject': {'x': -339.9358520507812...,...,CollisionObject1,2.0,-306.875488,8.193738,119.999870,1.0,44.938327,11.430179,False,Background
903957,915626,1048917,1754,3,1,1.647010e+09,1.647010e+09,1.647010e+09,0,"[{'hitPointOnObject': {'x': 168.2522430419922,...",...,road_base_network.004,1.0,168.252243,-6.553910,-242.814865,1.0,2.679853,2.716267,False,Background
753570,762697,895988,1754,2,3,1.646921e+09,1.646921e+09,1.646921e+09,3,[{'hitPointOnObject': {'x': 244.18141174316406...,...,terrain_H.001,2.0,227.358109,-1.423856,50.581993,1.0,31.681535,14.371960,False,Background


In [34]:
sample.groupby("Collider_Categorical")["ordinalOfHit"].count().sort_values()

Collider_Categorical
Passive_Agent                 17
Active_Agent                  24
TaskBuilding_Residential     122
Graffity                     193
TaskBuilding_Public          450
Building                    1282
Background                  2912
Name: ordinalOfHit, dtype: int64

In [20]:
patterns

{'\\d{2}_Sa': 'Passive_Agent',
 '\\d{2}_Cma': 'Active_Agent',
 'TaskBuilding_2': 'TaskBuilding_Public',
 'TaskBuilding_3': 'TaskBuilding_Public',
 'TaskBuilding_5': 'TaskBuilding_Public',
 'TaskBuilding_8': 'TaskBuilding_Public',
 'TaskBuilding_9': 'TaskBuilding_Public',
 'TaskBuilding_11': 'TaskBuilding_Public',
 'TaskBuilding_13': 'TaskBuilding_Public',
 'TaskBuilding_14': 'TaskBuilding_Public',
 'TaskBuilding_20': 'TaskBuilding_Public',
 'TaskBuilding_21': 'TaskBuilding_Public',
 'TaskBuilding_23': 'TaskBuilding_Public',
 'TaskBuilding_27': 'TaskBuilding_Public',
 'TaskBuilding_29': 'TaskBuilding_Public',
 'TaskBuilding_32': 'TaskBuilding_Public',
 'TaskBuilding_34': 'TaskBuilding_Public',
 'TaskBuilding_38': 'TaskBuilding_Public',
 'TaskBuilding_41': 'TaskBuilding_Public',
 'TaskBuilding_42': 'TaskBuilding_Public',
 'TaskBuilding_44': 'TaskBuilding_Public',
 'TaskBuilding_45': 'TaskBuilding_Public',
 'TaskBuilding_47': 'TaskBuilding_Public',
 'TaskBuilding_50': 'TaskBuilding_Public

In [14]:
the_rejected = temp_one_part_l[(temp_one_part_l.Collider_stays == False) & (temp_one_part_l["ordinalOfHit"] == 2)]

In [16]:
the_rejected.groupby("hitObjectColliderName")["ordinalOfHit"].count()

hitObjectColliderName
01_Cma                          4
02_Cma                         39
03_Cma                          1
04_Cma                         33
05_Cma                        197
                             ... 
terrain_V.001                6360
terrain_W.001                3700
terrain_X.001                5288
terrain_Y.001                1834
traffic_light_single_tall       2
Name: ordinalOfHit, Length: 1085, dtype: int64

In [ ]:
temp_one_part_l['newcol1'] = [temp_one_part_l['Eucledian_distance'].diff() if temp_one_part_l["ordinalOfHit"] == 2]


In [ ]:
temp_one_part_l.head(50)